# Описание задачи

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. \
\
Обучить модель - классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.
Необходимо построить модель со значением метрики качества F1 не меньше 0.75.

# Загрузка данных

In [1]:
pip install -q pymystem3

DEPRECATION: colab 1.13.5 has a non-standard dependency specifier pytz>=2011n. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of colab or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -q tqdm

DEPRECATION: colab 1.13.5 has a non-standard dependency specifier pytz>=2011n. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of colab or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install -q imblearn

DEPRECATION: colab 1.13.5 has a non-standard dependency specifier pytz>=2011n. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of colab or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import re
import string

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer  
stop_words = stopwords.words('english')

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import f1_score
from sklearn.utils import class_weight

from imblearn.over_sampling import RandomOverSampler

import warnings
warnings.filterwarnings('ignore')

/Users/alexander/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
#constants
RANDOM_STATE = 42

In [6]:
try:
    df = pd.read_csv('toxic_comments.csv')
except:
    df = pd.read_csv('/datasets/toxic_comments.csv')
df.head(3)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


# Предобработка данных

Удалим ненужный столбец

In [8]:
df = df.drop(columns='Unnamed: 0')
df.head(3)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0


In [9]:
df.duplicated().sum()

0

In [10]:
df['text'].duplicated().sum()

0

Дубликатов нет

In [11]:
df['toxic'].unique()

array([0, 1])

In [12]:
toxic_0 = df[df['toxic'] == 0]['toxic'].count()
toxic_1 = df[df['toxic'] == 1]['toxic'].count()
print('Доля с классом_0 = {}'.format(round(toxic_0 / df['toxic'].count(), 3)))
print('Доля с классом_1 = {}'.format(round(toxic_1 / df['toxic'].count(), 3)))

Доля с классом_0 = 0.898
Доля с классом_1 = 0.102


In [13]:
df.shape

(159292, 2)

In [14]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Обнаружен дисбаланс классов

# Подготовка данных

Создадим фунцию, которая подготавливает текст к обучению: 
- лемматизация 
- удаление стоп-слов
- удаление специальных знаков

In [15]:
sw = stopwords.words('english')
lemmatizer = WordNetLemmatizer() 

def clean_text(text):
    
    text = text.lower()
    
    text = re.sub(r"[^a-zA-Z?.!,¿]+", " ", text)

    text = re.sub(r"http\S+", "",text)
    
    html=re.compile(r'<.*?>') 
    
    text = html.sub(r'',text)
    
    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`" + '_'
    for p in punctuations:
        text = text.replace(p,'')
        
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    
    text = [lemmatizer.lemmatize(word) for word in text]
    
    text = " ".join(text)
    
    return text

In [16]:
df['text'] = df['text'].apply(lambda x: clean_text(x))
 
df.head()

,text,toxic
0,explanation edits made username hardcore metal...,0
1,aww match background colour seemingly stuck th...,0
2,"hey man, really trying edit war guy constantly...",0
3,make real suggestion improvement wondered sect...,0
4,"you, sir, hero chance remember page",0


Разделим датасет на обучающую, валидационную и тестовую выборки, а также сделаем векторизацию корпуса из текстов

In [17]:
X_train_valid, X_test , y_train_valid, y_test = train_test_split(
    df['text'].values,
    df['toxic'].values,
    test_size=0.1,
    random_state=RANDOM_STATE,
    stratify=df['toxic'].values)

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_valid,
    y_train_valid,
    test_size=0.2,
    random_state=RANDOM_STATE,
    stratify=y_train_valid)

print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)

print(y_train.shape)
print(y_valid.shape)
print(y_test.shape)

(114689,)
(28673,)
(15930,)
(114689,)
(28673,)
(15930,)


In [18]:
tfidf_vectorizer = TfidfVectorizer() 
features_train = tfidf_vectorizer.fit_transform(X_train)
features_valid = tfidf_vectorizer.transform(X_valid)
features_test = tfidf_vectorizer.transform(X_test)

print(features_train.shape)
print(features_valid.shape)
print(features_test.shape)

(114689, 143746)
(28673, 143746)
(15930, 143746)


# Обучение моделей

Так как на этапе предобработки данных мы обнаружили дисбаланс классов, необходимо учесть этот факт в ходе обучения моделей

In [19]:
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights

array([0.5565536 , 4.92058521])

Словарь весов

In [20]:
dft = {
    0: 0.556555078310615, 
    1: 4.920469522240527
}
dft

{0: 0.556555078310615, 1: 4.920469522240527}

## DecisionTreeClassifier

Так как подбор всех гиперпараметров решающего дерева занимает длительное время, то разделим этот процесс на два этапа

In [23]:
model_tree = DecisionTreeClassifier(random_state=RANDOM_STATE, class_weight=dft)
PARAMS_TREE = {
    'criterion': ['gini', 'entropy', 'log_loss'], 
    'splitter': ['best', 'random']
}

grid_tree = GridSearchCV(model_tree, PARAMS_TREE, cv=5, scoring='f1')
grid_tree.fit(features_train, y_train)

print('Best score = {}'.format(grid_tree.best_score_))
print('Best params:')
print(grid_tree.best_params_)

Best score = 0.6645109705130744
Best params:
{'criterion': 'entropy', 'splitter': 'random'}


Теперь подберем глубину решающего дерева

In [24]:
best_model = None
best_depth = None
best_f1 = 0
for depth in range(50, 601, 50):
    model_tree = DecisionTreeClassifier(random_state=RANDOM_STATE, 
                                    criterion='entropy', 
                                    splitter='random',
                                    max_depth=depth,
                                    class_weight=dft)

    model_tree.fit(features_train, y_train)
    predict_valid = model_tree.predict(features_valid)
    f1 = f1_score(y_valid, predict_valid)
    if f1 > best_f1:
        best_f1 = f1
        best_model = model_tree
        best_depth = depth

print('Best depth = {}'.format(best_depth))
print('Best best_f1 = {}'.format(best_f1))

Best depth = 600
Best best_f1 = 0.6937605396290051


### Valid

In [25]:
model_tree = DecisionTreeClassifier(random_state=RANDOM_STATE, 
                                    criterion='entropy', 
                                    splitter='random',
                                    max_depth=350,
                                    class_weight=dft)
model_tree.fit(features_train, y_train)
predict_valid = best_model.predict(features_valid)

f1_score(y_valid, predict_valid)

0.6937605396290051

**Вывод:** Качество модели хуже, чем предыдущая рассмотренная модель F1 = 0.7

## RandomForestClassifier

Используем кросс-валидацию для подбора гиперпараметров

### Cross valid

In [26]:
param_dist = {'n_estimators': range(50, 500, 50),
              'max_depth': range(1, 20)}

rf = RandomForestClassifier()

rand_search = RandomizedSearchCV(rf, 
                                 param_distributions = param_dist, 
                                 n_iter=5, 
                                 cv=5)

rand_search.fit(features_train, y_train)

best_rf = rand_search.best_estimator_

print('Best hyperparameters:',  rand_search.best_params_)

Best hyperparameters: {'n_estimators': 50, 'max_depth': 18}


### Valid

In [27]:
best_rf.fit(features_train, y_train)
predict_valid = best_rf.predict(features_valid)
f1 = f1_score(y_valid, predict_valid)

print('Best best_f1 = {}'.format(f1))

Best best_f1 = 0.0006863417982155114


**Вывод:** Модель **RandomForestClassifier** имеет очень низкое качество, поэтому ее не стоит выбирать как приоритетную

Проверим на тестовой выборке лучшую модель - LogisticRegression

In [29]:
#best_model = grid_log.best_estimator_
predict_test = model_tree.predict(features_test)
f1 = f1_score(y_test, predict_test)

print('Best best_f1 = {}'.format(f1))

Best best_f1 = 0.6960991835500454


**Вывод:** Качество модели удовлетворяет ТЗ и равно F1 = 0.69

# Вывод

Мы провели исследование и обучили несколько разных моделей классификации, после этого выявили, что DecisionTreeClassifier имеет наиболее высокое качество, где F1 = 0.69. Также ходе обучения мы учли наличие дисбаланса классов, и перед обучением подготовили текст.